In [1]:
%reload_ext autoreload
%autoreload 2


In [3]:
import numpy as np
from scipy import stats
from six.moves import urllib
import json
import pandas as pd
import math

from counter import var_counter
import sys

from Classes import DataO
from Classes import MapO


In [4]:
datapath="../3_Raw_Data/60_Monthly_MeanTemp_2022-2100_UK"
resultpath="../6_Results/full_map_test/"

In [7]:
'TODO:plot spatial distribution of values, 3D histogram?'

'TODO:plot spatial distribution of values, 3D histogram?'

In [6]:
#### RUNNING ANALYSIS ####
#** Generating the threshold table ***

thresh_df=pd.DataFrame(
    columns=["Maxtemp",
            "Mintemp",
            "Avgtemp",
            "Rain",
            "Wind",
            "Snow"],
    index= ["Thermoelectric",
            "Hydroelectric",
            "Wind",
            "Solar PV",
            "Transformers",
            "Poles",
            "Overhead Lines",
            "Demand"]
)

thresh_df['Maxtemp']=[22,35,35,35,35,35,35,35]
thresh_df['Mintemp']=[3,0,0,0,0,0,0,0]
thresh_df['Avgtemp']=[20,25,25,25,25,25,25,25]
thresh_df['Rain']=[5,50,50,50,50,50,50,50]
thresh_df['Wind']=[20,30,30,30,30,30,30,30]
thresh_df['Snow']=[1,50,50,50,50,50,50,50]

'TESTING'
print(thresh_df)

                Maxtemp  Mintemp  Avgtemp  Rain  Wind  Snow
Thermoelectric       22        3       20     5    20     1
Hydroelectric        35        0       25    50    30    50
Wind                 35        0       25    50    30    50
Solar PV             35        0       25    50    30    50
Transformers         35        0       25    50    30    50
Poles                35        0       25    50    30    50
Overhead Lines       35        0       25    50    30    50
Demand               35        0       25    50    30    50


In [94]:
### LOAD ALL WEAtHER VARIABLE DATA ###

spatial_res=5
time_res='monthly'

Maxtemp=DataO.DataObject(title="5_Monthly_Maxtemp_2021_2040_UK",
                        varname="Maxtemp",sres=spatial_res,tres=time_res,
                        fdname="5_Monthly_Maxtemp_2021_2040_UK",
                        fdpath="../3_Raw_Data/5_Monthly_Maxtemp_2021_2040_UK",
                        respath='../6_Results/data/')


Avgtemp=DataO.DataObject(title="5_Monthly_Avgtemp_2021_2040_UK",
                        varname="Avgtemp",sres=spatial_res,tres=time_res,
                        fdname="5_Monthly_Avgtemp_2021_2040_UK",
                        fdpath="../3_Raw_Data/5_Monthly_Avgtemp_2021_2040_UK",
                        respath='../6_Results/data/')

Mintemp=DataO.DataObject(title="5_Monthly_Mintemp_2021_2040_UK",
                        varname="Mintemp",sres=spatial_res,tres=time_res,
                        fdname="5_Monthly_Mintemp_2021_2040_UK",
                        fdpath="../3_Raw_Data/5_Monthly_Mintemp_2021_2040_UK",
                        respath='../6_Results/data/')

Rain=DataO.DataObject(title="5_Monthly_Rain_2021_2040_UK",
                        varname="Rain",sres=spatial_res,tres=time_res,
                        fdname="5_Monthly_Rain_2021_2040_UK",
                        fdpath="../3_Raw_Data/5_Monthly_Rain_2021_2040_UK",
                        respath='../6_Results/data/')

Snow=DataO.DataObject(title="5_Monthly_Snow_2021_2040_UK",
                        varname="Snow",sres=spatial_res,tres=time_res,
                        fdname="5_Monthly_Snow_2021_2040_UK",
                        fdpath="../3_Raw_Data/5_Monthly_Snow_2021_2040_UK",
                        respath='../6_Results/data/')

WeatherVariables=[Maxtemp,Mintemp,Avgtemp,Rain,Snow]

'TESTING'
for object in WeatherVariables:
    assert object.untreated_array.shape == (12,239,243,153), 'Loaded data array does not have expected shape'


In [95]:
### RUNNING STAT TESTS ###

Maxtemp.run_stats(KStest=False,stats=True,tp_90=True)
Mintemp.run_stats(KStest=False,stats=True,tp_90=True)
Avgtemp.run_stats(KStest=False,stats=True,tp_90=True)
Rain.run_stats(KStest=False,stats=True,tp_90=True)
Snow.run_stats(KStest=False,stats=True,tp_90=True)


In [96]:
'TODO: update with correct row as component'

### SETTING THRESHOLDS FOR EACH VARIABLE ###
for variable in WeatherVariables:
    threshold=thresh_df[variable.varname]["Thermoelectric"]
    variable.set_threshold(threshold)


'Testing'
for variable in WeatherVariables:
    assert variable.threshold==thresh_df[variable.varname]["Thermoelectric"]


In [109]:
### COUNTING EXCESS DAYS ###

Maxtemp.counter()
Mintemp.counter()
Avgtemp.counter()
Rain.counter()
Snow.counter()


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/_l/nh3jq3sx4k997zbcg87p0qbm0000gn/T/ipykernel_57467/3366500887.py", line 4, in <cell line: 4>
    Mintemp.counter()
  File "/Users/katerinafoka/Documents/Imperial_Courses/Master Thesis/7.Code/SEF_ElecResilience/5_Calculations/Classes/DataO.py", line 191, in counter
    assert self.p90_array.shape==(239,243,153), "p90_array passed on doesn't have correct shape"
AssertionError: p90_array passed on doesn't have correct shape

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 1993, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/Librar

In [98]:
### CREATING MAP ###

Map_5_Monthly_2021_2040=MapO.MapObject("Maxtemp - Excess days",Maxtemp,respath=resultpath)

In [99]:
Map_5_Monthly_2021_2040.build_props()

In [100]:
Map_5_Monthly_2021_2040.geojson_write('v1_test')

In [31]:
#Map of Max temperature
Maxtemp.max_val()
Maxtemp.max_array
Map_5_Monthly_2021_2040.geojson_props('Maxtemp - abs max',Maxtemp.flat_array(Maxtemp.max_array))
Map_5_Monthly_2021_2040.geojson_write('2021_2041')

In [85]:
##DEBUGGING DATA/SQUARES MISMATCH ###

test_Maxtemp=Maxtemp

[counter_array, fcounter_array]=test_Maxtemp.counter()

for i in range(fcounter_array.size-1):
    if i%243==0:
        fcounter_array[i]=30
    else:
        fcounter_array[i]=0

test_Map=MapO.MapObject("sqr number",test_Maxtemp,respath=resultpath)

test_Map.geojson_props('sqr id', fcounter_array)
test_Map.geojson_write('v2_debugging')



In [89]:
def clean_array(array):
    new_array=array
    for i in range(array.size-2):
        if i%243==0:
            new_array=np.delete(new_array,i)

for i in range(fcounter_array.size):
    fcounter_array[i]=i

clean_array(fcounter_array)
test_Map=MapO.MapObject("sqr id",test_Maxtemp,respath=resultpath)

test_Map.geojson_props('sqr id', fcounter_array)
test_Map.geojson_write('v3_debugging')

IndexError: index 37179 is out of bounds for axis 0 with size 37179

In [66]:
x_coord=test_Maxtemp.xcoord
y_coord=test_Maxtemp.ycoord
test_Maxtemp.fcounter_array.size

new_array=test_Maxtemp.fcounter_array

for i in range(test_Maxtemp.fcounter_array.size):
   if i%244==0:
    new_array=np.delete(new_array,i-1)

final=new_array

for i in range(new_array.size):
    if i%244==0:
        new_array[i]=30
    else:
        new_array[i]=0

test_Map=MapO.MapObject("sqr number",test_Maxtemp,respath=resultpath)

test_Map.geojson_props('sqr id', final)
test_Map.geojson_write('short_debugging')

In [108]:
Maxtemp=DataO.DataObject(title="5_Monthly_Maxtemp_2021_2040_UK",
                        varname="Maxtemp",sres=spatial_res,tres=time_res,
                        fdname="5_Monthly_Maxtemp_2021_2040_UK",
                        fdpath="../3_Raw_Data/5_Monthly_Maxtemp_2021_2040_UK",
                        respath='../6_Results/data/')

### RUNNING STAT TESTS ###

Maxtemp.run_stats(KStest=False,stats=True,tp_90=True)

### SETTING THRESHOLDS FOR EACH VARIABLE ###
Maxtemp.set_threshold(22)


'Testing'
for variable in WeatherVariables:
    assert variable.threshold==thresh_df[variable.varname]["Thermoelectric"]

### COUNTING EXCESS DAYS ###

Maxtemp.counter()

### CREATING MAP ###

Map_5_Monthly_2021_2040=MapO.MapObject("Maxtemp - Excess days",Maxtemp,respath=resultpath)
Map_5_Monthly_2021_2040.build_props()
Map_5_Monthly_2021_2040.geojson_write('v4_test')


(12, 239, 243, 153)
